# Textklassifikation mit RNN: GermEval 2018

*GermEval* – für German Evaluation – ist ein jährlicher Wettbewerb im Bereich Natural Language Processing für deutschsprachige Texte (s. [https://germeval.github.io/](https://germeval.github.io/)).

Im Jahr 2018 ging es um die Erkennung von Beleidigungen in deutschsprachigen Tweets.

In dieser Aufgabe wollen wir Rekurrente Neuronale Netze (RNN) zur Klassifikation nutzen. Zunächst einmal starten wir mit Vorarbeiten.

## Format der Daten

Die Trainings- und Testdaten liegen als mit Tabulatoren separierte Textdateien (Tab Separated Values – TSV) vor. Uns interessieren die erste Spalte (der Tweet) und die zweite Spalte (`OFFENSE` für Beleidigung bzw. `OTHER` für keine Beleidigung).

In [ ]:
! head -10 texts/germeval2018.training.tsv

## Lesen der Daten

Für das Einlesen der Datensätze verwenden wir die Klasse `NamedTuple`, mit der sich die Daten einfach speichern lassen.

In [1]:
from collections import namedtuple

Record = namedtuple('Record', [ 'text', 'primary_label', 'secondary_label' ])

with open('texts/germeval2018.training.tsv', 'r') as file:
    training_data = [ Record(*line[:-1].split('\t')) for line in file ]

with open('texts/germeval2018.test.tsv', 'r') as file:
    test_data = [ Record(*line[:-1].split('\t')) for line in file ]

training_data[0:5]

[Record(text='@corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?', primary_label='OTHER', secondary_label='OTHER'),
 Record(text='@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.', primary_label='OTHER', secondary_label='OTHER'),
 Record(text='@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️', primary_label='OTHER', secondary_label='OTHER'),
 Record(text='@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!', primary_label='OTHER', secondary_label='OTHER'),
 Record(text='@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.', primary_label='OFFENSE', secondary_label='INSULT')]

## Überblick über die Daten

Wir schauen uns die Verteilung der Kategorien in den Trainings- und Testdaten an.

In [2]:
from collections import Counter

Counter([ (record.primary_label, record.secondary_label) for record in training_data ])

Counter({('OTHER', 'OTHER'): 3321,
         ('OFFENSE', 'INSULT'): 595,
         ('OFFENSE', 'PROFANITY'): 71,
         ('OFFENSE', 'ABUSE'): 1022})

In [3]:
Counter([ (record.primary_label, record.secondary_label) for record in test_data ])

Counter({('OTHER', 'OTHER'): 2330,
         ('OFFENSE', 'ABUSE'): 773,
         ('OFFENSE', 'INSULT'): 381,
         ('OFFENSE', 'PROFANITY'): 48})

In [ ]:
!pip install torchtext spacy
!python -m spacy download de_core_news_md

## Preprocessing der Tweets

Für die weitere Verarbeitung wollen wir Twitter Handles (`@username`) löschen und das Hashtag-Zeichen entfernen. Damit verhindern wir, dass unser Model später die Namen auswendig lernt, um die Daten zu klassifizieren. 

In [4]:
import re

def clean_tweet(text):
    """ Preprocess and tokenize a tweet. """
    
    # remove handles, i.e. @username
    text = re.sub('\@\w+', '', text)
    # remove hashtags, quotes, etc.
    text = re.sub('[\#"\']+', '', text)
    text = text.replace('-', ' ')
    return text

clean_tweet(training_data[4].text)

' kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen   schickt diese Stümper SPD in die Versenkung.'

## Vektorisierung mit vortrainierten Wortvektoren

Wir nutzen vortrainierte Wortvektoren aus Spacy.

In [5]:
import torch
import spacy
import numpy as np

nlp = spacy.load("de_core_news_md")

def vectorize(text):
    """Vectorize text using the German SpaCy tokenizer"""
    return torch.Tensor(np.array([tok.vector for tok in nlp(clean_tweet(text)) if tok.has_vector ]))


In [6]:
vectorize(training_data[0].text).shape

torch.Size([16, 300])

## Laden der Daten

Mithilfe der Funktion `vectorize()` definieren wir die Funktion `collate_batch()`, die einen Batch in zwei Tensoren – für die Label und die Texte – umwandelt.
Damit wir das RNN später effizient trainieren können, bringen wir die Text-Tensoren mithilfe der Funktion `pad_sequence()` auf die gleiche Länge. 

In [21]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

LABEL = { 'OFFENSE': 1, 'OTHER': 0 }

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    
    for record in batch:
        label_list.append(LABEL[record.primary_label])
        processed_text = vectorize(record.text)
        text_list.append(processed_text)
        lengths.append(processed_text.shape[0])
    return torch.tensor(label_list), pad_sequence(text_list, batch_first=True), lengths

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, num_workers=5, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=5, collate_fn=collate_batch)

In [22]:
labels, texts, lengths = next(iter(train_dataloader))
labels.shape, texts.shape, len(lengths)

(torch.Size([64]), torch.Size([64, 49, 300]), 64)

## Klassifikation von Text mittels RNNs

Texte bestehen aus einer *Folge* von Wörtern. 
Rekurrente Neuronale Netze (RNNs) eignen sich gut für die Verarbeitung von Folgen.

Unser Netz wird dabei aus zwei Schichten bestehen:
1. das eigentliche RNN aus *Long-Short-Term-Memoy (LSTM)* Zellen oder *Gated Recurrent Units (GRU)*, die die Wortfolge auf eine Folge von *Zuständen* abbilden,
2. einen linearen Layer, der den letzten Zustand auf eine eindimensionale Variable abbildet.


### Aufbau des RNNs

Nun bauen wir das oben beschriebene Netz aus Embedding Layer, RNN Layer und Linear Layer auf.

Die Funktionen `torch.nn.utils.rnn.pack_padded_sequence` und `torch.nn.utils.rnn.pad_packed_sequence` packen bzw. entpacken die Tensoren für eine effiziente Berechnung.

In [36]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class RNN(torch.nn.Module) :
    def __init__(self, hidden_dim = 20, embedding_dim = 300, dropout = 0.4) :
        super().__init__()
  
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, dropout=0.1, batch_first=True)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=3, dropout=dropout, batch_first=True, bidirectional=False)
        self.linear = nn.Linear(hidden_dim, 2)
    
        
    def forward(self, _x, **kwargs):
        (x, input_lengths) = _x
        
        x = pack_padded_sequence(x, input_lengths, batch_first=True, enforce_sorted=False)
        #x, (ht, ct) = self.lstm(x)
        x, ht = self.gru(x)
        x, output_lengths = pad_packed_sequence(x, batch_first=True)
        #print(ht[-1].shape)
        return self.linear(ht[-1])

### Training und Validierung

Der Einfachheit halber verwenden wir für Training und Validierung weiterhin Scikit Learn und binden das RNN als `skorch.NeuralNetBinaryClassifier` ein.

In [37]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [38]:
model = RNN(hidden_dim=64, dropout=0.5)

In [39]:
count_parameters(model)

120322

In [40]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer=torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from tqdm.notebook import tqdm

epochs = 10 
training_loss = []
testing_loss = []
training_acc = []
testing_acc = []


with tqdm(range(epochs)) as iterator:
    for epoch in iterator:

        train_loss = 0.0
        train_acc = 0     
        for idx, (target, text, length) in enumerate(train_dataloader):

            target, text = target.to(device), text.to(device)

            optimizer.zero_grad() 
            output = model((text, length))

            loss = loss_fn(output, target) 
            loss.backward() 
            optimizer.step()
            
            train_loss += loss_fn(output, target).item()
            predictions = output.data.max(1)[1]
            train_acc += (predictions == target).sum().item()
 
        training_acc.append(train_acc/len(train_dataloader.dataset))
        training_loss.append(train_loss/len(train_dataloader.dataset))
            
        test_loss = 0
        test_acc = 0
        with torch.no_grad():
            for target, text, length in test_dataloader:
                target, text = target.to(device), text.to(device)
                output = model((text, length))
                loss = loss_fn(output, target)
                prediction = torch.argmax(output, 1)
                test_acc += (prediction == target).sum().item()
                test_loss += loss.item()        
                
            testing_acc.append(test_acc/len(test_dataloader.dataset))
            testing_loss.append(test_loss/len(test_dataloader.dataset))
            
        loss = running_loss/count
        accuracy = 100. * running_correct/count 
        iterator.set_postfix_str(f"train_acc: {train_acc/len(train_dataloader.dataset):.2f} test_acc: {test_acc/len(test_dataloader.dataset):.2f} train_loss: {train_loss/len(train_dataloader.dataset):.2e} test_loss: {test_loss/len(test_dataloader.dataset):.2e}")

  0%|          | 0/10 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fdd13ddedd0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7fdd13ddedd0>if w.is_alive():

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError    : self._shutdown_workers()can only test a child process

  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/op